In [3]:
!pip install datasets
!apt-get install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [1]:
import pandas as pd
import requests
from datasets import load_dataset
import string

import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer

from transformers import pipeline, tokenization_utils

#dataset
ds = load_dataset('yelp_review_full')

trained = ds['train']
tested = ds['test']

train_df = trained.to_pandas()
test_df = tested.to_pandas()
combined_df = pd.concat([train_df, test_df])

#preprocessing
def remove_punctuation(text):
  for punctuation in string.punctuation:
    text = text.replace(punctuation, '')
  return text

combined_df['text'] = combined_df['text'].apply(remove_punctuation)

#tokenize
combined_df['text'] = combined_df['text'].apply(nltk.word_tokenize)

#sample
nltk.download('stopwords')
combined_df_sample = combined_df.sample(n=1000)
#lowercase
combined_df_sample['text'] = combined_df_sample['text'].apply(lambda x: [word.lower() for word in x])
#stopwords_remove
combined_df_sample['text'] = combined_df_sample['text'].apply(lambda x: [word for word in x if word not in nltk.corpus.stopwords.words('english')])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#pipelines
classifier1 = pipeline(
    task = 'sentiment-analysis',
    model = 'distilbert-base-uncased',
)
classifier2 = pipeline(
    task = 'sentiment-analysis',
    model = 'aychang/roberta-base-imdb',
)

#results
prediction_classifier1 = []
prediction_classifier2 = []
labels_true = []

#review loop and pass text to get the sentiment and save results in lists created
for i, review in enumerate(combined_df_sample['text']):
  try:
    prediction_classifier1.append(classifier1(review))
    prediction_classifier2.append(classifier2(review))
    labels_true.append(combined_df_sample['label'][i])
  except:
    continue

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]